In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from datetime import datetime, timedelta
import pandas as pd

current_date = pd.to_datetime(datetime.utcnow()).floor('H') # - timedelta(hours=1)
print(f'{current_date=}')
# current_date = pd.Timestamp('2023-02-28 09:00:00')

current_date=Timestamp('2024-01-04 02:00:00')


In [3]:
import hopsworks
import src.config as config
from src.config import FEATURE_VIEW_METADATA
from src.feature_store_api import get_feature_store, get_or_create_feature_view
feature_view = get_or_create_feature_view(FEATURE_VIEW_METADATA)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/305300
Connected. Call `.close()` to terminate connection gracefully.
2024-01-03 18:05:00,768 INFO: Feature view already exists, skipping creation.
Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/305300
Connected. Call `.close()` to terminate connection gracefully.


In [18]:
fetch_data_from = current_date - timedelta(days=28)
fetch_data_to = current_date - timedelta(hours=1)
print(f'{fetch_data_from=}')
print(f'{fetch_data_to=}')

fetch_data_from=Timestamp('2023-12-07 02:00:00')
fetch_data_to=Timestamp('2024-01-04 01:00:00')


In [19]:
fetch_data_to = pd.to_datetime(fetch_data_to, utc=True).floor('H')
fetch_data_from = pd.to_datetime(fetch_data_from, utc=True).floor('H')

In [20]:
ts_data = feature_view.get_batch_data(
        start_time=pd.to_datetime((fetch_data_from - timedelta(days=1)), utc=True).floor('H'),
        end_time=pd.to_datetime((fetch_data_to + timedelta(days=1)), utc=True).floor('H')
    )

ts_data= ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]
ts_data.sort_values(by=['pickup_location_id', 'pickup_hour'], inplace=True)
location_ids = ts_data['pickup_location_id'].unique()
print(f'{ts_data.pickup_hour.min()=}')
print(f'{ts_data.pickup_hour.max()=}')
print(f'{(ts_data.pickup_hour.max() - ts_data.pickup_hour.min()).days=}')

Finished: Reading data from Hopsworks, using ArrowFlight (82.21s) 
ts_data.pickup_hour.min()=Timestamp('2023-12-07 02:00:00+0000', tz='UTC')
ts_data.pickup_hour.max()=Timestamp('2024-01-04 01:00:00+0000', tz='UTC')
(ts_data.pickup_hour.max() - ts_data.pickup_hour.min()).days=27


In [26]:
print(f'{len(ts_data.reset_index())=}')
print(f'{len(location_ids)=}')

len(ts_data.reset_index())=175914
len(location_ids)=262


In [22]:
print(f'{(ts_data.pickup_hour.max() - ts_data.pickup_hour.min()).total_seconds() / 3600 =}')

(ts_data.pickup_hour.max() - ts_data.pickup_hour.min()).total_seconds() / 3600 =671.0


In [34]:
ts_data['pickup_hour'].value_counts() < 262

pickup_hour
2023-12-07 02:00:00+00:00    False
2023-12-23 11:00:00+00:00    False
2023-12-23 13:00:00+00:00    False
2023-12-23 14:00:00+00:00    False
2023-12-23 15:00:00+00:00    False
                             ...  
2024-01-01 22:00:00+00:00     True
2024-01-01 21:00:00+00:00     True
2024-01-01 20:00:00+00:00     True
2024-01-01 19:00:00+00:00     True
2024-01-04 01:00:00+00:00     True
Name: count, Length: 672, dtype: bool

In [40]:
missing_locations = ts_data.loc[ts_data['pickup_hour'] == '2024-01-01 22:00:00+00:00', :]['pickup_location_id'].unique()

In [44]:
import numpy as np
np.setdiff1d(location_ids, missing_locations)

array([176, 199])

In [50]:
ts_data.loc[ts_data.pickup_location_id == 176, :]

,pickup_hour,rides,pickup_location_id,pickup_ts
106774,2023-12-07 02:00:00+00:00,0,176,1701914400000
69254,2023-12-07 03:00:00+00:00,0,176,1701918000000
117401,2023-12-07 04:00:00+00:00,0,176,1701921600000
79622,2023-12-07 05:00:00+00:00,0,176,1701925200000
35908,2023-12-07 06:00:00+00:00,0,176,1701928800000
...,...,...,...,...
161488,2023-12-31 18:00:00+00:00,0,176,1704045600000
161673,2023-12-31 19:00:00+00:00,0,176,1704049200000
161925,2023-12-31 20:00:00+00:00,0,176,1704052800000
162422,2023-12-31 21:00:00+00:00,0,176,1704056400000


In [ ]:
print(f'{config.N_FEATURES=}')
print(f'{(config.N_FEATURES * len(location_ids)) =}'))

In [52]:
full_range = pd.date_range(ts_data['pickup_hour'].min(), ts_data['pickup_hour'].max(), freq='H')
full_range

DatetimeIndex(['2023-12-07 02:00:00+00:00', '2023-12-07 03:00:00+00:00',
               '2023-12-07 04:00:00+00:00', '2023-12-07 05:00:00+00:00',
               '2023-12-07 06:00:00+00:00', '2023-12-07 07:00:00+00:00',
               '2023-12-07 08:00:00+00:00', '2023-12-07 09:00:00+00:00',
               '2023-12-07 10:00:00+00:00', '2023-12-07 11:00:00+00:00',
               ...
               '2024-01-03 16:00:00+00:00', '2024-01-03 17:00:00+00:00',
               '2024-01-03 18:00:00+00:00', '2024-01-03 19:00:00+00:00',
               '2024-01-03 20:00:00+00:00', '2024-01-03 21:00:00+00:00',
               '2024-01-03 22:00:00+00:00', '2024-01-03 23:00:00+00:00',
               '2024-01-04 00:00:00+00:00', '2024-01-04 01:00:00+00:00'],
              dtype='datetime64[ns, UTC]', length=672, freq='H')

In [53]:
rides_i = ts_data.loc[ts_data.pickup_location_id == 176, ['pickup_hour', 'rides']]
            
# quick way to add missing dates with 0 in a Series
# taken from https://stackoverflow.com/a/19324591
rides_i.set_index('pickup_hour', inplace=True)
rides_i.index = pd.DatetimeIndex(rides_i.index)
rides_i = rides_i.reindex(full_range, fill_value=0)

# add back `location_id` columns
rides_i['pickup_location_id'] = 176

rides_i.reset_index(inplace=True)

In [58]:
len(rides_i)

672

In [62]:
len(ts_data.loc[ts_data.pickup_location_id == 176, ['pickup_hour', 'rides']])

597

In [ ]:
x = ts_data.groupby(['pickup_location_id', 'pickup_ts']).size().reset_index(name='count')
len(x)

In [ ]:
from src.inference import load_batch_of_features_from_store

features = load_batch_of_features_from_store(current_date)

In [ ]:
# import hopsworks
# hopsworks.connection().close()